# Unscented transform

The unscented transform is a statistical method used to approximate a probability distribution for a random variable. The unscented transform generates a set of "sigma points" that are selected to capture the key features of the distribution, such as mean, covariance, and higher-order moments. These sigma points are then propagated through a nonlinear function to obtain a transformed distribution. The transformed sigma points are then used to approximate the mean and covariance of the transformed distribution.

In the unscented transform, alpha, beta, and kappa are parameters that control the spread of the sigma points around the mean of the distribution. They are usually selected based on the desired properties of the unscented transform, such as accuracy, robustness, and computational efficiency.

- Alpha: Controls the spread of the sigma points around the mean of the distribution. A smaller alpha results in a tighter spread, which can lead to a more accurate estimate but also be more sensitive to outliers. A larger alpha results in a wider spread, which can lead to a more robust estimate but also be less accurate. Common values of alpha are between 0 and 1, with values of 0.001, 0.01, or 0.1 being common.

- Beta: Controls the incorporation of higher-order statistical moments into the unscented transform. A value of 2 is commonly used for Gaussian distributions, which incorporates the mean and covariance of the distribution. A value greater than 2 can be used to incorporate higher-order moments, while a value less than 2 can be used to reduce the computational cost of the unscented transform.

- Kappa: Controls the scaling of the sigma points. A value of 0 means that the sigma points are not scaled, while a positive value scales the spread of the sigma points to account for higher-dimensional distributions or to achieve a desired trade-off between accuracy and computational efficiency. A negative value of kappa is sometimes used to adjust the scaling of the sigma points in the unscented transform. However, its use is not as common as a positive value of kappa and its effect is not well understood.
A negative value of kappa can cause the sigma points to be drawn from outside the distribution, which can result in unexpected behavior. Therefore, it is generally recommended to use a positive value of kappa or a value of 0 to avoid potential problems.

The optimal values of alpha, beta, and kappa depend on the specific problem and distribution, and may need to be determined through trial and error or by using established guidelines.

In [156]:
import numpy as np
import matplotlib.pyplot as plt

In [157]:
def van_der_merwe_weights(size, alpha, kappa, beta):
  lmbd = alpha ** 2 * (size + kappa) - kappa
  wi = 1 / (2 * (size + lmbd))
  wm = np.full(2 * size + 1, wi)
  wc = np.full(2 * size + 1, wi)
  wm[0] = lmbd / (size + lmbd)
  wc[0] = wm[0] + (1 - alpha **2 - beta)
  return wm, wc

def unscented_transform(transform_functor, mean, cov, mean_weights, cov_weights, alpha, kappa):
  # Input dimension
  L = mean.shape[0]

  # Sigma
  lmbd = alpha ** 2 * (L + kappa) - kappa
  sigma = np.sqrt(L * lmbd)

  # Decompose covariance matric cov = L @ L.T
  P = np.linalg.cholesky(cov)

  # Generate sigma-points
  sigma_points = np.zeros((2 * L + 1, L))
  sigma_points[0, :] = mean
  for i in range(0, L):
    sigma_points[i + 1, :] = mean + sigma * P[:, i]
    sigma_points[L + i + 1, :] = mean - sigma * P[:, i]

  # Apply transformation to the sigma points
  transformed_sigma_points = transform_functor(sigma_points)

  # Compute transformation mean
  mean_tr = transformed_sigma_points.T @ mean_weights

  # Compute transformation covariance
  centered_transformed_sigma_points = transformed_sigma_points - mean_tr
  cov_tr = centered_transformed_sigma_points.T @ np.diag(cov_weights) @ centered_transformed_sigma_points

  return mean_tr, cov_tr

In [158]:
def cartesian_to_polar(x):
    y = np.zeros(x.shape)
    y[:, 0] = np.hypot(x[:, 0], x[:, 1])
    y[:, 1] = np.arctan2(x[:, 1], x[:, 0])
    return y

# Input mean and covariancer
mean = np.array([12.3, 7.6])
cov = np.array([[1.44, 0.0], [0.0, 2.89]])
print("input mean =", mean)
print("input covariance =\n", cov)

# Unscented transform parameters
alpha = 1e-2
kappa = -1
beta = 0
input_size = mean.shape[0]

# Unscented transform weights
wm, wc = van_der_merwe_weights(input_size, alpha, kappa, beta)

# Apply unscented transform
tr_mean, tr_cov = unscented_transform(cartesian_to_polar, mean, cov, wm, wc, alpha, kappa)
print("transformation mean =", tr_mean)
print("transformation covariance =\n", tr_cov)

input mean = [12.3  7.6]
input covariance =
 [[1.44 0.  ]
 [0.   2.89]]
transformation mean = [14.51616072  0.55146333]
transformation covariance =
 [[1.22125441 0.02861947]
 [0.02861947 0.0080347 ]]
